In [7]:
%pip install lobsang openai python-dotenv jsonschema

Note: you may need to restart the kernel to use updated packages.


# Directives

Directives in lobsang can be used to guide a LLM to produce specific output formats. For example, you can use a directive to ask the LLM to produce a JSON object. To do so, a directive generally does two things:

1. It embeds / wraps a user message with instructs the LLM. For example, the `JSONDirective` adds a provided JSON schema together with general instructions to the user message.
   ```python
   original = "Create a marvel hero"
   #  ☝️ becomes 👇 
   embedded = """Create a marvel hero
   
    Create a JSON object with the following schema:
    ```json
    {
        <SCHEMA HERE>      
    }
   """
    ```
2. It tries to parse the LLM output to the desired format. For example, the `JSONDirective` tries to parse the LLM output to a JSON object using the provided schema. It will also validate the output against the schema.
   **Note:** If the LLM output is not valid JSON, the directive will not raise an error but add it to the returned `info` object. This is so processing multiple message won't stop if one message fails. (⚠️ May change in the future!)


Now let's give it a try! 🚀 But first we have to import some stuff and set up our environment (make sure to update the `.env` file with your own OpenAI API key, see [1_basics](1_basics.ipynb) for more details).

In [8]:
import os

from dotenv import load_dotenv

from lobsang import Chat, OpenAI
from lobsang.directives import JSONDirective

load_dotenv()

# Load OpenAI API key from .env file (please update .env file with your own API key)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
assert OPENAI_API_KEY, "Please set OPENAI_API_KEY in .env file"

f"All set! 🎉 Let's get started! 🚀 OPENAI_API_KEY={OPENAI_API_KEY[:15]}..."

"All set! 🎉 Let's get started! 🚀 OPENAI_API_KEY=sk-PbRY4M6AH6Xh..."

In our example, we want to create a marvel hero and output it as a JSON object. To do so, we have to create a JSON schema for our hero. We need to use the [JSON Schema](https://json-schema.org/) standard for that. The schema below describes a hero with a name, age and powers 🦸

In [9]:
hero_schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string", "description": "The name of the hero"},
        "age": {"type": "integer", "minimum": 0, "description": "The age of the hero"},
        "powers": {
            "type": "array",
            "items": {"type": "string"},
            "description": "The powers of the hero"
        }
    }
}

Next, we'll take a look at embedding a user message with the `JSONDirective`. The `JSONDirective` takes the schema as an argument and embeds it into the user message:

In [10]:
hero_message = "Create a marvel hero"
hero_directive = JSONDirective(schema=hero_schema)

embedded_message, _ = hero_directive.embed(hero_message)

print(embedded_message)

Create a marvel hero

Create a JSON object with the following schema:
```json
{'type': 'object', 'properties': {'name': {'type': 'string', 'description': 'The name of the hero'}, 'age': {'type': 'integer', 'minimum': 0, 'description': 'The age of the hero'}, 'powers': {'type': 'array', 'items': {'type': 'string'}, 'description': 'The powers of the hero'}}}
```


️️️☝️ As you can see the instructions are added to the user message. When using a json directive in chat, note that the user message is updated with the instructions. This is important to know when using the chat history for further processing (we'll see this in a bit).

👇 For now, let's also take a look at how the `JSONDirective` parses the LLM output.

In [11]:
# Hypothetical LLM output
response = """
Sure, i can create a marvel hero for you.

Here you go:
```json
{
    "name": "Spiderman",
    "age": 18,
    "powers": ["spider sense", "web slinging", "superhuman strength"]
}
```

I hope you like it!
"""

parsed, info = hero_directive.parse(response)
print(parsed)

```json
{
    "name": "Spiderman",
    "age": 18,
    "powers": [
        "spider sense",
        "web slinging",
        "superhuman strength"
    ]
}
```


️️️☝️ As you can see, the `JSONDirective` parsed the LLM output to a JSON object and returned a string with a json block, removing the surrounding text. By default, the pruned response is returned as a string (see above), but you can change this behavior by setting `prune=False`. This can be helpful if you want to keep the surrounding text of the LLM output.

👇 However, this string representation is not very useful for further processing. Therefore, the `JSONDirective` also returns the parsed object as a python object in the `info` dictionary. Let's take a look at it:

In [12]:
info

{'original': '\nSure, i can create a marvel hero for you.\n\nHere you go:\n```json\n{\n    "name": "Spiderman",\n    "age": 18,\n    "powers": ["spider sense", "web slinging", "superhuman strength"]\n}\n```\n\nI hope you like it!\n',
 'directive': JSONDirective,
 'json': {'name': 'Spiderman',
  'age': 18,
  'powers': ['spider sense', 'web slinging', 'superhuman strength']},
 'error': None,
 'schema': {'type': 'object',
  'properties': {'name': {'type': 'string',
    'description': 'The name of the hero'},
   'age': {'type': 'integer',
    'minimum': 0,
    'description': 'The age of the hero'},
   'powers': {'type': 'array',
    'items': {'type': 'string'},
    'description': 'The powers of the hero'}}}}

️️️☝️ In the info dictionary, you can find the parsed json object under `json`. You can also find the original message under `original`. In our case, parsing was successful, so `error` is `None`. If parsing fails, the `error` key will contain the error message.

**Note:** If an error occurs, the json directive will always return the original message. No matter whether `prune` is set to `True` or `False`. This is because since parsing failed, there is no point in pruning the message and replacing it with `None` or an error message would break the chat history.

👇 Ok, so now we know how to embed a user message and parse the LLM output. It's time to put it all together and create a chat with a `JSONDirective`:

In [13]:
# Create a chat instance with OpenAI LLM
chat = Chat(llm=OpenAI(api_key=OPENAI_API_KEY))

# Initialize the JSONDirective again (we could also reuse the one from above, we will use the same schema though)
hero_directive = JSONDirective(schema=hero_schema)

messages = [
    "Create a marvel hero",
    hero_directive,  # 👈 We use the JSONDirective from above (the chat instance will use the directive for the corresponding user message, i.e. the message one index before)
]

# Let's chat!
chat.run(messages)

print(chat)

USER: Create a marvel hero

Create a JSON object with the following schema:
```json
{'type': 'object', 'properties': {'name': {'type': 'string', 'description': 'The name of the hero'}, 'age': {'type': 'integer', 'minimum': 0, 'description': 'The age of the hero'}, 'powers': {'type': 'array', 'items': {'type': 'string'}, 'description': 'The powers of the hero'}}}
```

ASSISTANT: ```json
{
    "name": "Iron Man",
    "age": 40,
    "powers": [
        "Superhuman strength",
        "Flight",
        "Energy blasts"
    ]
}
```


️️️️☝️ As you can see, the chat history contains the modified user message with the instructions and the parsed LLM output. Since `replace` is set to `True`, the original message is replaced with the parsed message. 

👇 However, we can take a look at the original message in the `info` dictionary:

In [14]:
# Let's get the assistant message from the chat history (i.e. the LLM's response)
assistant_message = chat[-1]

# Get the info dictionary from the assistant message
info = assistant_message.info 

# Get the original message from the info dictionary
original_message = info["original"]

print(original_message)

Okay, here is a JSON object representing a Marvel hero:

```json
{
  "name": "Iron Man",
  "age": 40,
  "powers": [
    "Superhuman strength",
    "Flight",
    "Energy blasts"
  ]
}
```


️️️☝️ Here the LLM's response may also have some text above and below the json block (**Note:** This can vary from run to run). Any surrounding text was pruned by the `JSONDirective` when parsing the LLM's response (because `prune` was set to `True`) which is why the message in the chat history only contains the json block.

## Conclusion

Congratulations! 🎉 You just created your first chat with a `JSONDirective`! 🚀 

In this notebook, we learned how to embed a user message with a `JSONDirective` and how to parse the LLM's response. We also learned how to create a chat with a `JSONDirective` and how to access the original message in the chat history. 

**Note:** In this example we used OpenAI's LLM. However, the `JSONDirective` can be used with any LLM. You can also adapt the instructions of a directive via the `instructions` argument. 

If you have any question to hesitate to reach out to us on [Discord](https://discord.gg/wMHVAaqh).
If you've found a bug, a spelling mistake or suggestions on what could be improved, please open an issue or a pull request on [GitHub](https://github.com/cereisen/lobsang).

See you in the next part! 👋 We hope we got you hooked 🎣 